# PATIENT MOVEMENT ANALYSIS FOR PARKINSON'S DISEASE SEVERITY PREDICTION

### This project is a video classification model for Parkinson's disease severity using gait analysis. 

## SETUP

In [74]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
import os
import cv2

## LOAD KEYPOINTS

This function loads the keypoints from the JSON files. It returns the keypoints as a list.<br>
<br>
The keypoints have been extracted using the AlphaPose pose estimator and the Halpe Full-Body Human Keypoints Dataset.

In [75]:
def load_keypoints_from_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    keypoints_list = []
    for element in data:
        keypoints = np.array(element['keypoints']).reshape(-1, 3)
        keypoints_list.append(keypoints)

    return keypoints_list

## FEATURE EXTRACTION

The feature extraction function takes in a list of keypoints as a parameter. <br>

The features extracted for Parkinson's disease severity prediction are the following: <br>

- Distance between the nose and the approximate center of the feet, indicating posture.
- Average confidence score of the wrists, indicating arm steadiness (tremors).
- Average distance between the nose and the wrists, indicating arm swing.
- Ratio of the distances between the nose and the left and right wrists, indicating arm swing symmetry.
- Maximum height of the ankles, indicating step height.
- Distance between the left and right ankles, indicating step length.

In [76]:
def extract_features(keypoints_list):
    total_nose_to_foot_distance = 0
    total_wrist_steadiness = 0
    total_nose_to_wrist_distance = 0
    total_arm_swing_symmetry = 0
    max_ankle_height = 0
    total_step_length = 0
    num_frames = 0

    #indices of keypoints
    nose_idx = 0
    left_wrist_idx = 9
    right_wrist_idx = 10
    left_ankle_idx = 15
    right_ankle_idx = 16

    for keypoints in keypoints_list:
        if keypoints.shape[0] < 17:
            continue
       
        nose = keypoints[nose_idx]
        left_wrist = keypoints[left_wrist_idx]
        right_wrist = keypoints[right_wrist_idx]
        left_ankle = keypoints[left_ankle_idx]
        right_ankle = keypoints[right_ankle_idx]

        # features 
        nose_to_foot_distance = np.linalg.norm(nose[:2] - ((left_ankle[:2] + right_ankle[:2]) / 2))
        wrist_steadiness = np.mean([left_wrist[2], right_wrist[2]])
        nose_to_wrist_distance = np.mean([np.linalg.norm(nose[:2] - left_wrist[:2]), np.linalg.norm(nose[:2] - right_wrist[:2])])
        arm_swing_symmetry = np.abs(np.linalg.norm(nose[:2] - left_wrist[:2]) / np.linalg.norm(nose[:2] - right_wrist[:2]))
        max_ankle_height = np.max([left_ankle[1], right_ankle[1]])
        step_length = np.linalg.norm(left_ankle[:2] - right_ankle[:2])

        #Update Values
        total_nose_to_foot_distance += nose_to_foot_distance
        total_wrist_steadiness += wrist_steadiness
        total_nose_to_wrist_distance += nose_to_wrist_distance
        total_arm_swing_symmetry += arm_swing_symmetry
        max_ankle_height = max(max_ankle_height, np.max([left_ankle[1], right_ankle[1]]))
        total_step_length += step_length
        num_frames += 1

    if num_frames > 0:
        avg_nose_to_foot_distance = total_nose_to_foot_distance / num_frames
        avg_wrist_steadiness = total_wrist_steadiness / num_frames
        avg_nose_to_wrist_distance = total_nose_to_wrist_distance / num_frames
        avg_arm_swing_symmetry = total_arm_swing_symmetry / num_frames
        avg_step_length = total_step_length / num_frames
    else:
        return None
    
    return np.array([
        avg_nose_to_foot_distance,
        avg_wrist_steadiness,
        avg_nose_to_wrist_distance,
        avg_arm_swing_symmetry,
        max_ankle_height,
        avg_step_length
    ])

## LOAD FEATURES

This function loads the keypoints from all the json files and calls the extract_features() function on each file. <br>

It returns a list of lists of features and a list of their respective labels.

In [77]:
def load_features(data_dir):
    features = []
    labels = []

    for severity in ['NORMAL', 'MILD', 'MODERATE', 'SEVERE']:
        dir_path = os.path.join(data_dir, severity)

        for json_file in os.listdir(dir_path):
            file_path = os.path.join(dir_path, json_file)
            keypoints = load_keypoints_from_json(file_path)
            #filtered_keypoints = filter_keypoints_by_confidence(keypoints)

            if len(keypoints) > 0:
                feature = extract_features(keypoints)
                if feature is not None:
                    features.append(feature)
                labels.append(severity)

    return np.array(features), np.array(labels)

Create model function

In [78]:
def create_model():
    model = Sequential()
    model.add(Dense(64, input_dim=6, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

features, labels = load_features('./JSON/')
label_dict = {'NORMAL': 0, 'MILD': 1, 'MODERATE': 2, 'SEVERE': 3}

Train the model

In [79]:
# One-hot encode the labels
label_dict = {'NORMAL': 0, 'MILD': 1, 'MODERATE': 2, 'SEVERE': 3}
labels = labels.flatten().astype(str) 
labels = [label_dict[l] for l in labels]
labels = tf.keras.utils.to_categorical(labels)
labels_one_hot = np.array(labels)
# Split data into training sets and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels_one_hot, test_size=0.2, random_state=42)


# Create model
model = create_model()

# Train the model
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2703 - loss: 54.2350 - val_accuracy: 0.5424 - val_loss: 19.4028
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3442 - loss: 41.6895 - val_accuracy: 0.5424 - val_loss: 19.4581
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3515 - loss: 42.7195 - val_accuracy: 0.5424 - val_loss: 14.7524
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3653 - loss: 33.7529 - val_accuracy: 0.5085 - val_loss: 10.8361
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3583 - loss: 29.2573 - val_accuracy: 0.4915 - val_loss: 8.7254
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3670 - loss: 21.5512 - val_accuracy: 0.4407 - val_loss: 8.0100
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3255 - loss: 19.8770 - val_accuracy: 0.4746 - val_loss: 8.6526
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3870 - loss: 20.9828 - val_accuracy: 0.4746 - val_loss: 8.1852
Epoch 9

Evaluate the model

In [80]:
# Evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

# Predict the testing set
y_pred = np.argmax(model.predict(X_test), axis=-1)
y_test = np.argmax(y_test, axis=-1)

# Print classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

CNN Error: 52.54%
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x31c146b60> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
              precision    recall  f1-score   support

           0       0.72      0.66      0.69        32
           1       0.00      0.00      0.00         7
           2       0.23  

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

## Video Analysis with OpenCV